In [1]:
!nvidia-smi

Tue Sep 24 19:51:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8             27W /  420W |     494MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

device(type='cuda')

In [16]:
pip install -U ultralytics

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


# 경로

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 특정 경로 설정 (OneDrive의 바탕 화면 경로)
desktop_path = r"C:/Users/User/Desktop/YOLO_0924"

# 데이터셋 경로 설정
image_dir = os.path.join(desktop_path, 'yolodataset/images')
label_dir = os.path.join(desktop_path, 'yolodataset/labels')

train_image_dir = os.path.join(desktop_path, 'dataset/train/images')
train_label_dir = os.path.join(desktop_path, 'dataset/train/labels')
val_image_dir = os.path.join(desktop_path, 'dataset/val/images')
val_label_dir = os.path.join(desktop_path, 'dataset/val/labels')
test_image_dir = os.path.join(desktop_path, 'dataset/test/images')
test_label_dir = os.path.join(desktop_path, 'dataset/test/labels')

# 필요한 디렉토리 생성
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(test_image_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)

# split train, test, val

In [5]:
# 이미지 파일 리스트와 라벨 파일 리스트 얻기
image_files = sorted(os.listdir(image_dir))
label_files = sorted(os.listdir(label_dir))

# 이미지 파일 이름과 라벨 파일 이름에서 확장자 제거
image_files_set = set([os.path.splitext(f)[0] for f in image_files])
label_files_set = set([os.path.splitext(f)[0] for f in label_files])

# 공통된 파일 리스트 생성
common_files = list(image_files_set & label_files_set)

# 이미지 파일과 라벨 파일의 전체 파일 경로를 생성
image_files = [f + os.path.splitext(image_files[0])[1] for f in common_files]
label_files = [f + os.path.splitext(label_files[0])[1] for f in common_files]

# 이미지가 없는 라벨 파일 처리
if label_files:
    label_ext = os.path.splitext(label_files[0])[1]
else:
    print("라벨 파일이 없습니다.")
    label_ext = ""

if label_ext:
    no_image_labels = [f + label_ext for f in (label_files_set - image_files_set)]
else:
    no_image_labels = []

# 데이터셋을 학습, 검증, 테스트로 나누기 (예: 70% 학습, 20% 검증, 10% 테스트)
train_images, temp_images, train_labels, temp_labels = train_test_split(
    image_files, label_files, test_size=0.3, random_state=42
)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.33, random_state=42
) # 0.33 * 0.3 = 0.1

# 이미지가 없는 라벨 파일 처리 (비어있는 경우 처리 건너뜀)
if no_image_labels:
    train_labels_no_image, temp_labels_no_image = train_test_split(
        no_image_labels, test_size=0.3, random_state=42
    )
    val_labels_no_image, test_labels_no_image = train_test_split(
        temp_labels_no_image, test_size=0.33, random_state=42
    ) # 0.33 * 0.3 = 0.1
else:
    train_labels_no_image, val_labels_no_image, test_labels_no_image = [], [], []

# 파일 이동 함수
def move_files(file_list, src_dir, dst_dir):
    for file_name in file_list:
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
        else:
            print(f"파일이 존재하지 않습니다: {src_path}")

# 학습 데이터 이동
move_files(train_images, image_dir, train_image_dir)
move_files(train_labels, label_dir, train_label_dir)
move_files(train_labels_no_image, label_dir, train_label_dir)

# 검증 데이터 이동
move_files(val_images, image_dir, val_image_dir)
move_files(val_labels, label_dir, val_label_dir)
move_files(val_labels_no_image, label_dir, val_label_dir)

# 테스트 데이터 이동
move_files(test_images, image_dir, test_image_dir)
move_files(test_labels, label_dir, test_label_dir)
move_files(test_labels_no_image, label_dir, test_label_dir)



# YOLO 학습

In [7]:
from ultralytics import YOLO
import os 

# YOLOv8n 모델 로드

model = YOLO('yolov8l.yaml')

# yaml 파일 로드
yaml_file = "C:/Users/User/Desktop/YOLO_0924/datasetV3.yaml"


# 모델 학습
model.train(data=yaml_file, epochs=300)

# 학습된 모델 평가
results = model.val()

# 모델 저장 경로 설정
model_save_dir = 'C:/Users/User/Desktop/YOLO_0924/model'
os.makedirs(model_save_dir, exist_ok=True)
model_save_path = os.path.join(model_save_dir, 'best.pt')

# 모델 저장 
if hasattr(model, 'ckpt') and model.ckpt is not None:
    model.save(model_save_path)
    print(f"모델이 성공적으로 저장되었습니다: {model_save_path}")
else:
    print("체크포인트가 없습니다. 모델이 제대로 학습되었는지 확인하세요.")

New https://pypi.org/project/ultralytics/8.2.100 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.63  Python-3.12.4 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine\trainer: task=detect, mode=train, model=yolov8l.yaml, data=C:/Users/User/Desktop/YOLO_0924/datasetV3.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

100%|██████████| 6.25M/6.25M [00:00<00:00, 11.9MB/s]


AMP: checks passed 


train: Scanning C:\Users\User\Desktop\YOLO_0924\dataset\train\labels... 7414 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7414/7414 [00:03<00:00, 1975.25it/s]


train: New cache created: C:\Users\User\Desktop\YOLO_0924\dataset\train\labels.cache


val: Scanning C:\Users\User\Desktop\YOLO_0924\dataset\val\labels... 2129 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2129/2129 [00:01<00:00, 1212.33it/s]


val: New cache created: C:\Users\User\Desktop\YOLO_0924\dataset\val\labels.cache
Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/464 [00:00<?, ?it/s]

# Confusion Matrix

## Confusion Matrix

In [ ]:
import seaborn as sn  # Seaborn 라이브러리 추가
import numpy as np
import matplotlib.pyplot as plt
import os


#classes
classes = [
    
]

# Confusion Matrix 저장 및 해상도 조정
def plot_confusion_matrix(matrix, save_path, class_names):
    # 값이 0인 곳은 NaN으로 바꿔서 빈칸으로 표시
    array = np.where(matrix == 0, np.nan, matrix)
    
    fig, ax = plt.subplots(figsize=(20, 20))  # 그림 크기를 20x20으로 설정
    sn.heatmap(array, annot=True, cmap='Blues', fmt='.0f', square=True, vmin=0.0, xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 8}).set_facecolor((1, 1, 1))
    
    ax.set_xlabel('True Labels')
    ax.set_ylabel('Predicted Labels')
    ax.set_title('Confusion Matrix')
    
    # 고해상도로 저장 (dpi=400)
    plt.savefig(save_path, dpi=400)
    plt.close()

# Confusion matrix 데이터 가져오기
conf_matrix = results.confusion_matrix.matrix  # matrix 속성으로 접근하여 confusion matrix를 numpy 배열로 변환

# confusion matrix를 저장할 경로 설정
conf_matrix_save_path = os.path.join(model_save_dir, 'confusion_matrix_high_res.png')

# confusion matrix 플로팅 및 저장
plot_confusion_matrix(conf_matrix, conf_matrix_save_path, classes)
print(f"Confusion matrix가 저장되었습니다: {conf_matrix_save_path}")

## filltered_ConfusionMatrix

### import

In [ ]:
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import os

### set Class

In [ ]:
# 모든 클래스 목록
classes = [
    
]

# 포함할 클래스 목록
include_classes = [
    
]

# 제외할 클래스 목록 생성
exclude_classes = [cls for cls in classes if cls not in include_classes]

### 실행

In [ ]:
# Confusion Matrix 저장 및 해상도 조정
def plot_confusion_matrix(matrix, save_path, class_names, dpi):
    # 값이 0인 곳은 NaN으로 바꿔서 빈칸으로 표시
    array = np.where(matrix == 0, np.nan, matrix)
    
    # 그림 크기를 조정하여 클래스 이름이 잘 보이도록 설정
    fig, ax = plt.subplots(figsize=(20, 20))  # 그림 크기를 20x20으로 설정하여 텍스트 간격 확보
    sn.heatmap(array, annot=True, cmap='coolwarm', fmt='.0f', square=True, vmin=0.0, xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 8}).set_facecolor((1, 1, 1))
    
    ax.set_xlabel('True Labels', fontsize=12)
    ax.set_ylabel('Predicted Labels', fontsize=12)
    ax.set_title('Confusion Matrix', fontsize=15)
    
    # x축과 y축의 텍스트 크기 조정
    ax.xaxis.set_tick_params(labelsize=10)
    ax.yaxis.set_tick_params(labelsize=10)
    
    # 해상도 조정
    plt.tight_layout()  # 여백을 자동으로 조정
    plt.savefig(save_path, dpi=dpi)
    plt.close()


#Confusion Matrix를 fillterd Confusion Matrix로 변환
def convert_Matrix():
    # Confusion matrix 데이터 가져오기
    conf_matrix = results.confusion_matrix.matrix  # matrix 속성으로 접근하여 confusion matrix를 numpy 배열로 변환
    
    # 제외할 클래스의 인덱스 추출
    exclude_indices = [classes.index(cls) for cls in exclude_classes]
    
    # 해당 인덱스를 사용하여 행과 열 제거
    filtered_conf_matrix = np.delete(conf_matrix, exclude_indices, axis=0)  # 행 제거
    filtered_conf_matrix = np.delete(filtered_conf_matrix, exclude_indices, axis=1)  # 열 제거
        
    # exclude_classes에 해당하는 클래스 목록만 남김
    filtered_classes = [cls for i, cls in enumerate(classes) if i not in exclude_indices]

    return filtered_conf_matrix, filtered_classes


def main():
    # confusion matrix를 저장할 경로 설정
    conf_matrix_save_path = os.path.join(model_save_dir, 'confusion_matrix_filtered.png')
    
    # 필터링된 confusion matrix 플로팅 및 저장 ***dpi(해상도) 설정
    plot_confusion_matrix(filtered_conf_matrix, conf_matrix_save_path, filtered_conf_matrix, dpi=1600)
    print(f"Filtered confusion matrix가 저장되었습니다: {conf_matrix_save_path}")


main()